In [ ]:
# # !pip install -U langchain pypdfium2
# %pip install streamlit --upgrade
# %pip install streamlit-authenticator


In [1]:
import streamlit as st
import streamlit_authenticator as stauth


In [2]:
from streamlit_authenticator.utilities.hasher import Hasher

In [3]:
# import streamlit_authenticator as stauth
# hashed_passwords = stauth.Hasher(['streamlit']).generate()
# hashed_passwords

In [26]:
from streamlit_authenticator.utilities.hasher import Hasher

# Create an instance of the Hasher class with a list of passwords
passwords = ['password1', 'password250', ''] ## add your pwd examples
hasher = Hasher(passwords)

# Call the generate method on the Hasher instance
hashed_passwords = hasher.generate()

print(hashed_passwords)

['$2b$12$q.eUqMDZLuMjs1EdmT50O.K01JfrQQRTDY9C37Tf8uR6sqK1Xom6m', '$2b$12$x3h0ixsvoh5K1cv1aXAa3enfwlOVAtZyttSt3BrZqxg0O3gfqjyYK', '$2b$12$TunvzL7FNTEuMarCc4oQ.uOZBnRF8u2mAOoDbdDYlh1sQ9IClsxM6']


In [21]:
from langchain.document_loaders import PyPDFDirectoryLoader, PyPDFLoader, PyPDFium2Loader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.chains.question_answering import load_qa_chain
from langchain_community.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.memory import ConversationBufferMemory
import boto3, json, ast, logging, time, re

import re
import time
from glob import glob
from tqdm import tqdm



In [ ]:

from langchain_community.vectorstores import FAISS


In [6]:
docs = []
for file_name in tqdm(glob('./Data/*')):
    loader = PyPDFium2Loader(file_name)
    doc_pages = loader.load()
    docs += doc_pages

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]C:\ProgramData\anaconda3\envs\lang-pip\Lib\site-packages\pypdfium2\_helpers\textpage.py:81: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 10.11it/s]


In [7]:
len(docs)

34

In [8]:
x_docs = [doc for doc in docs if len(doc.page_content)>10]

In [9]:
from langchain.embeddings import BedrockEmbeddings

In [10]:
# Initialize the boto3 session and lambda client
session = boto3.Session(profile_name=, region_name='us-east-1') ## add your profile

In [11]:
model_id = "amazon.titan-embed-text-v1"
be = BedrockEmbeddings(
    model_id=model_id,
    credentials_profile_name=, region_name="us-east-1"
    
) ## add your profile

In [12]:
faiss_index = FAISS.from_documents(x_docs, be)

In [13]:
faiss_index.save_local("vectorstore")

In [14]:
from langchain_community.chat_models import BedrockChat
from typing import Any, Dict, Optional
from botocore.client import Config

bedrock_config = Config(connect_timeout=180, read_timeout=180, retries={'max_attempts': 5})
bedrock_client = session.client('bedrock-runtime')

bedrock_model_id = "anthropic.claude-3-haiku-20240307-v1:0"  # Bedrock model_id

model_kwargs_claude = {
    "temperature": 0.0,
    "top_p": 0.99,
    "max_tokens": 3000,
    "stop_sequences": ["\n\nHuman"]
}


llm =  BedrockChat(model_id=bedrock_model_id, 
                   client=bedrock_client,
                   model_kwargs=model_kwargs_claude)

In [15]:
prompt_template = """You are a helpful assistant that answers questions directly and only using the information provided in the context below. 
    Guidance for answers:
        - Always use English as the language in your responses.
        - In your answers, always use a professional tone.
        - Begin your answers with "Based on the context provided: "
        - Simply answer the question clearly and with lots of detail using only the relevant details from the information below. If the context does not contain the answer, say "Sorry, I didn't understand that. Could you rephrase your question?"
        - Use bullet-points and provide as much detail as possible in your answer. 
        - Always provide a summary at the end of your answer.
        
    Now read this context below and answer the question at the bottom.
    
    Context: {context}

    Question: {question}
    
    think step by step and answer the question with utmost precision and accuracy.
    
    Assistant:
    """

In [20]:
 """ 
 
 You are a knowledgeable AI assistant named AWS Support AI, specializing in AWS services and solutions. Your primary role is to assist AWS employees by providing accurate and concise guidance on AWS offerings, including but not limited to Amazon Aurora, Amazon Bedrock, and other AWS managed services
 
 """

' \n\nYou are a knowledgeable AI assistant named AWS Support AI, specializing in AWS services and solutions. Your primary role is to assist AWS employees by providing accurate and concise guidance on AWS offerings, including but not limited to Amazon Aurora, Amazon Bedrock, and other AWS managed services\n\n'

In [16]:

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=faiss_index.as_retriever(
        search_type="similarity", search_kwargs={"k": 5}
    ),
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        # "memory": ConversationBufferMemory(memory_key="history", input_key="question"),
    }
)

In [18]:
qa.invoke({"query": query})

{'query': 'What is Aurora Standard and Aurora I/O-Optimized?',
 'result': 'Based on the context provided:\n\nAurora offers two configuration options for optimizing your database spend:\n\nAurora Standard:\n- A cost-effective configuration for applications with low to moderate I/O usage\n- You pay for database instances, storage, and pay-per-request I/O operations\n\nAurora I/O-Optimized:\n- Delivers improved price performance for I/O-intensive applications\n- Ideal for workloads with high I/O variability or where I/O spend exceeds 25% of total Aurora costs\n- Offers predictable pricing with no charges for read and write I/O operations\n\nKey differences:\n- Aurora Standard charges for I/O operations, while Aurora I/O-Optimized does not\n- Aurora I/O-Optimized is better suited for I/O-intensive workloads to provide cost savings\n\nSummary:\n- Aurora Standard is a cost-effective configuration for low to moderate I/O usage\n- Aurora I/O-Optimized is optimized for I/O-intensive workloads a

In [19]:
query = "What is Aurora Standard and Aurora I/O-Optimized?"
result = qa.invoke({"query": query})
print(result['result'])

Based on the context provided:

Aurora offers two configuration options for optimizing your database spend:

Aurora Standard:
- A cost-effective configuration for applications with low to moderate I/O usage
- You pay for database instances, storage, and pay-per-request I/O operations

Aurora I/O-Optimized:
- Delivers improved price performance for I/O-intensive applications
- Ideal for workloads with high I/O variability or where I/O spend exceeds 25% of total Aurora costs
- Offers predictable pricing with no charges for read and write I/O operations

Key differences:
- Aurora Standard charges for I/O operations, while Aurora I/O-Optimized does not
- Aurora I/O-Optimized is better suited for I/O-intensive workloads to provide cost savings

In summary:
- Aurora Standard is a cost-effective option for applications with low to moderate I/O needs
- Aurora I/O-Optimized is designed for I/O-intensive workloads to provide improved price performance and predictable costs


In [41]:
qa.combine_documents_chain.memory

In [42]:
qa({"query":"And what is its  utilities like pg_dump/pg_restore ?"})

{'query': 'And what is its  utilities like pg_dump/pg_restore ?',
 'result': 'Based on the context provided:\n\n- The context mentions that to migrate from PostgreSQL to Aurora PostgreSQL-Compatible Edition, you can use the standard pg_dump utility to export data from PostgreSQL and the pg_restore utility to import data into Aurora PostgreSQL.\n\n- Specifically, the context states:\n\n"If you want to migrate from PostgreSQL to Aurora and the other way around, you have several options:\n• You can use the standard pg_dump utility to export data from PostgreSQL and pg_restore utility to import data to Aurora, and the other way around."\n\nIn summary:\n\n- pg_dump is the standard PostgreSQL utility used to export data from a PostgreSQL database.\n- pg_restore is the standard PostgreSQL utility used to import data into a PostgreSQL database, including an Aurora PostgreSQL-Compatible Edition database.\n- These utilities allow you to migrate data between a PostgreSQL database and an Aurora Po

---

In [43]:

_template = """ Given the following conversation and a follow up question, rephrase the follow up question to be a 
standalone question without changing the content in given question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question: Assistant:"""

condense_question_prompt_template = PromptTemplate.from_template(_template)
qa_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [44]:
from langchain.chains import LLMChain

In [45]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain 

In [47]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
question_generator = LLMChain(llm=llm, prompt=condense_question_prompt_template, memory=memory)
doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=qa_prompt)
qa_chain = ConversationalRetrievalChain(
    retriever=faiss_index.as_retriever(search_kwargs={'k': 5}),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    memory=memory,
)

In [48]:
chat_history = []

In [49]:
question = 'what are agents for amazon bedrock?'
result = qa_chain({'question': question, 'chat_history': chat_history})
response = result['answer']
chat_history.append((question, response))
response

"Based on the context provided:\n\nAgents for Amazon Bedrock are:\n\n- Fully managed capabilities that make it easier for developers to create generative AI-based applications that can complete complex tasks for a wide range of use cases.\n\n- They automatically break down tasks and create an orchestration plan without any manual coding.\n\n- The agents securely connect to company data through an API, automatically converting data into a machine-readable format, and augmenting the request with relevant information to generate the most accurate response.\n\n- Agents can then automatically call APIs to fulfill a user's request, such as tracking inventory levels, sales data, supply chain information, and recommending optimal reorder points and quantities.\n\n- As fully managed capabilities, agents for Amazon Bedrock remove the undifferentiated lifting of managing system integration and infrastructure provisioning, allowing developers to use generative AI to its full extent throughout thei

In [50]:
result = qa_chain({'question': "And what is the definition of an agents for amazon bedrock?", 'chat_history': chat_history})
result['answer']

"Based on the context provided:\n\nAgents for Amazon Bedrock are defined as:\n\n- Fully managed capabilities that make it easier for developers to create generative AI-based applications\n- They can complete complex tasks for a wide range of use cases and deliver up-to-date answers based on proprietary knowledge sources\n- With just a few clicks, agents for Amazon Bedrock automatically break down tasks and create an orchestration plan, without any manual coding\n- The agent securely connects to company data through an API, automatically converting data into a machine-readable format, and augmenting the request with relevant information to generate the most accurate response\n- Agents can then automatically call APIs to fulfill a user's request\n- As fully managed capabilities, agents for Amazon Bedrock remove the undifferentiated lifting of managing system integration and infrastructure provisioning, allowing developers to use generative AI to its full extent throughout their organizat

In [51]:
chat_history

[('what are agents for amazon bedrock?',
  "Based on the context provided:\n\nAgents for Amazon Bedrock are:\n\n- Fully managed capabilities that make it easier for developers to create generative AI-based applications that can complete complex tasks for a wide range of use cases.\n\n- They automatically break down tasks and create an orchestration plan without any manual coding.\n\n- The agents securely connect to company data through an API, automatically converting data into a machine-readable format, and augmenting the request with relevant information to generate the most accurate response.\n\n- Agents can then automatically call APIs to fulfill a user's request, such as tracking inventory levels, sales data, supply chain information, and recommending optimal reorder points and quantities.\n\n- As fully managed capabilities, agents for Amazon Bedrock remove the undifferentiated lifting of managing system integration and infrastructure provisioning, allowing developers to use gener